In [ ]:
# =============================================================================
# Lexical Competition Pipeline: v20.5
# =============================================================================

!pip install kiwipiepy g2pk jamo python-Levenshtein tqdm > /dev/null
!apt-get -y install fonts-noto-cjk > /dev/null

import os
import re
import requests
import pandas as pd
import numpy as np
import Levenshtein
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
import seaborn as sns
import unicodedata
import pickle
from io import StringIO
from collections import Counter
from scipy import stats
from kiwipiepy import Kiwi
from g2pk import G2p
from jamo import h2j, j2hcj
from google.colab import drive
from tqdm.auto import tqdm

# --- 1. Setup ---
if not os.path.exists('/content/drive'):
    drive.mount('/content/drive')

BASE_DIR = '/content/drive/MyDrive/Korean_Lexical_Analysis_v14'
for d in ['inputs', 'outputs', 'cache']:
    os.makedirs(os.path.join(BASE_DIR, d), exist_ok=True)
INPUT_DIR = os.path.join(BASE_DIR, 'inputs')
OUTPUT_DIR = os.path.join(BASE_DIR, 'outputs')
CACHE_DIR = os.path.join(BASE_DIR, 'cache')

# Font
font_path = '/usr/share/fonts/opentype/noto/NotoSansCJK-Regular.ttc'
if os.path.exists(font_path):
    fm.fontManager.addfont(font_path)
    plt.rcParams['font.family'] = 'Noto Sans CJK JP'
    plt.rcParams['axes.unicode_minus'] = False

kiwi = Kiwi()
g2p = G2p()

# --- 2. Data Processing Helpers ---

def normalize_text(text):
    if not isinstance(text, str): return ""
    text = unicodedata.normalize('NFC', text)
    text = re.sub(r'[^가-힣\s]', '', text)
    return text.strip()

def get_phonological_form(orth):
    try: return j2hcj(h2j(g2p(orth)))
    except: return ""

def load_and_lemmatize_corpus():
    cache_path = os.path.join(CACHE_DIR, 'lemma_freq_full.pkl')
    if os.path.exists(cache_path):
        print(">>> [Cache Hit] Loading Lemma Frequency List...")
        return pd.read_pickle(cache_path)

    print(">>> [Cache Miss] Downloading & Lemmatizing Corpus (Kiwi)...")
    url = "https://raw.githubusercontent.com/hermitdave/FrequencyWords/master/content/2018/ko/ko_50k.txt"
    response = requests.get(url)
    if response.status_code != 200: raise Exception("Download failed")

    df_raw = pd.read_csv(StringIO(response.text), sep=' ', names=['wordform', 'count'])
    df_raw['wordform'] = df_raw['wordform'].astype(str).apply(normalize_text)
    df_raw = df_raw[df_raw['wordform'].str.len() > 0]

    lemma_counts = Counter()
    batch_size = 1000
    forms = df_raw['wordform'].tolist()
    counts = df_raw['count'].tolist()

    for i in tqdm(range(0, len(forms), batch_size), desc="Lemmatizing"):
        batch_forms = forms[i:i+batch_size]
        batch_counts = counts[i:i+batch_size]
        for form, count in zip(batch_forms, batch_counts):
            try:
                res = kiwi.analyze(form, top_n=1)
                if res: lemma_counts[res[0][0][0].form] += count
                else: lemma_counts[form] += count
            except: pass

    df_lemma = pd.DataFrame(lemma_counts.items(), columns=['orth', 'count'])
    total = df_lemma['count'].sum()
    df_lemma['frequency'] = (df_lemma['count'] / total) * 1_000_000
    df_lemma = df_lemma.sort_values('frequency', ascending=False).reset_index(drop=True)
    df_lemma = df_lemma.iloc[:50000].copy()

    print(f">>> Saving Cache to {cache_path}")
    df_lemma.to_pickle(cache_path)
    return df_lemma

def process_dataframe(df, label, freq_map=None):
    print(f"[{label}] Preprocessing...")
    if 'orth' not in df.columns and 'Word' in df.columns: df = df.rename(columns={'Word': 'orth'})
    df['orth_clean'] = df['orth'].astype(str).apply(normalize_text)
    if freq_map is not None:
        df['frequency'] = df['orth_clean'].map(freq_map).fillna(0.0)

    if 'form' not in df.columns:
        tqdm.pandas(desc=f"[{label}] G2P")
        df['form'] = df['orth_clean'].progress_apply(get_phonological_form)

    # POS Detection
    pos_candidates = ['POS', 'P.O.S', 'Part of Speech', '품사', 'pos', 'Class', 'Category']
    found_pos = next((c for c in pos_candidates if c in df.columns), None)
    if found_pos:
        df['pos_raw'] = df[found_pos]

    df = df.sort_values('frequency', ascending=False)

    agg_dict = {'orth_clean': 'first', 'frequency': 'sum'}
    if 'pos_raw' in df.columns: agg_dict['pos_raw'] = 'first'

    df_collapsed = df.groupby('form').agg(agg_dict).reset_index().rename(columns={'orth_clean': 'orth'})

    nonzero = df_collapsed[df_collapsed['frequency'] > 0]['frequency']
    floor = nonzero.min() if not nonzero.empty else 1e-9
    df_collapsed['frequency'] = df_collapsed['frequency'].replace(0.0, floor)

    df_collapsed = df_collapsed.sort_values('frequency', ascending=False).reset_index(drop=True)
    print(f"[{label}] Final N={len(df_collapsed)}")
    return df_collapsed

def get_base_lexicons():
    df_f_raw = load_and_lemmatize_corpus()
    df_f = process_dataframe(df_f_raw, "Full Lemma")
    freq_map = df_f.set_index('orth')['frequency'].to_dict()

    possible_paths = [
        os.path.join(INPUT_DIR, 'basic_vocab_5965.csv'),
        '/content/basic_vocab_5965.csv',
        '/content/drive/MyDrive/Korean_Lexical_Analysis_v14/inputs/basic_vocab_5965.csv'
    ]
    input_path = next((p for p in possible_paths if os.path.exists(p)), None)

    if input_path is None:
        raise FileNotFoundError(f"CRITICAL ERROR: 'basic_vocab_5965.csv' not found.")

    print(f">>> Loading Restricted Lexicon from: {input_path}")
    df_r = process_dataframe(pd.read_csv(input_path), "Restricted", freq_map=freq_map)
    return df_r, df_f

def sample_stratified(source_df, target_df, n_bins=20, seed=None):
    if seed is not None: np.random.seed(seed)
    target_freqs = np.log10(target_df['frequency'] + 1e-10)
    source_freqs = np.log10(source_df['frequency'] + 1e-10)
    bins = np.linspace(target_freqs.min(), target_freqs.max(), n_bins + 1)
    target_bins = np.digitize(target_freqs, bins)
    source_df = source_df.copy()
    source_df['bin'] = np.digitize(source_freqs, bins)
    target_counts = pd.Series(target_bins).value_counts().sort_index()
    sampled_indices = []
    for bin_idx, count in target_counts.items():
        candidates = source_df[source_df['bin'] == bin_idx]
        if len(candidates) == 0: continue
        sampled = candidates.sample(n=min(len(candidates), count), replace=False)
        sampled_indices.extend(sampled.index.tolist())
    return source_df.loc[sampled_indices].drop(columns=['bin']).reset_index(drop=True)

def calculate_metrics_single(df, source_corpus_df=None):
    forms = df['form'].values
    freqs = df['frequency'].values
    n = len(df)

    if source_corpus_df is not None:
        corpus_forms = source_corpus_df['form'].values
        corpus_freqs = source_corpus_df['frequency'].values
    else:
        corpus_forms = forms
        corpus_freqs = freqs

    corpus_map = {}
    for f, fr in zip(corpus_forms, corpus_freqs):
        l = len(f)
        if l not in corpus_map: corpus_map[l] = []
        corpus_map[l].append((f, fr))

    nds, snfs = np.zeros(n, dtype=int), np.zeros(n, dtype=float)

    for i in range(n):
        t = forms[i]
        candidates = []
        for l in [len(t)-1, len(t), len(t)+1]:
            if l in corpus_map: candidates.extend(corpus_map[l])
        for cand_f, cand_freq in candidates:
            if t == cand_f: continue
            if Levenshtein.distance(t, cand_f) <= 1:
                nds[i] += 1; snfs[i] += cand_freq

    df['ND'] = nds; df['SNF'] = snfs
    df['FWCS'] = df['frequency'] * df['ND'] * df['SNF']
    return df

# --- 3. Analysis Class ---

class LexicalAnalyzer:
    def __init__(self, df_r, df_f, n_resamples=100):
        self.df_r = df_r; self.df_f = df_f; self.n_resamples = n_resamples

        self.stats_cache_path = os.path.join(CACHE_DIR, 'resampling_stats_v20.pkl')
        self.dist_stats = self._load_stats()

        self.df_rand_a_plot = None; self.df_rand_b_plot = None
        self.res_r = None; self.res_top = None; self.res_full = None

    def _load_stats(self):
        if os.path.exists(self.stats_cache_path):
            with open(self.stats_cache_path, 'rb') as f:
                return pickle.load(f)
        return {
            'Random-A': {'zero_fwcs': [], 'gini': [], 'mean_nd': [], 'n_samples': []},
            'Random-B': {'zero_fwcs': [], 'gini': [], 'mean_nd': [], 'n_samples': []}
        }

    def _save_stats(self):
        with open(self.stats_cache_path, 'wb') as f:
            pickle.dump(self.dist_stats, f)

    def run_baselines(self):
        print("\n--- 1. Computing Baselines ---")

        # Restricted
        cache_r = os.path.join(CACHE_DIR, 'res_r.pkl')
        if os.path.exists(cache_r):
            self.res_r = pd.read_pickle(cache_r)
        else:
            print("   -> Computing Restricted...")
            self.res_r = calculate_metrics_single(self.df_r.copy())
            self.res_r.to_pickle(cache_r)

        # Freq-Top
        cache_top = os.path.join(CACHE_DIR, 'res_top.pkl')
        if os.path.exists(cache_top):
            self.res_top = pd.read_pickle(cache_top)
        else:
            print("   -> Computing Freq-Top...")
            n_target = len(self.df_r)
            self.res_top = calculate_metrics_single(self.df_f.sort_values('frequency', ascending=False).iloc[:n_target].copy())
            self.res_top.to_pickle(cache_top)

        # Full Lexicon
        cache_full = os.path.join(CACHE_DIR, 'res_full.pkl')
        if os.path.exists(cache_full):
            print("   -> [Cache Hit] Loaded Full Lexicon")
            self.res_full = pd.read_pickle(cache_full)
            # Validation: Ensure it has necessary columns. If stale cache, recompute.
            if 'ND' not in self.res_full.columns:
                print("      [Warning] Cached Full Lexicon invalid (missing ND). Recomputing...")
                self.res_full = calculate_metrics_single(self.df_f.copy())
                self.res_full.to_pickle(cache_full)
        else:
            print("   -> Computing Full Lexicon (Required for Fig 2)...")
            try:
                self.res_full = calculate_metrics_single(self.df_f.copy())
                self.res_full.to_pickle(cache_full)
            except Exception as e:
                print(f"   [Error] Could not compute Full Lexicon: {e}")
                self.res_full = None

    def run_resampling(self):
        print(f"\n--- 2. Resampling (Target N={self.n_resamples}) ---")
        n_target = len(self.df_r)

        # --- Random-A ---
        current_len_a = len(self.dist_stats['Random-A']['mean_nd'])
        remaining_a = self.n_resamples - current_len_a

        if remaining_a > 0:
            print(f"   -> Resuming Random-A: {current_len_a} done, {remaining_a} to go.")
            for i in tqdm(range(remaining_a), desc="Random-A"):
                df_sample = self.df_f.sample(n=n_target).copy()
                if i == 0 and current_len_a == 0: self.df_rand_a_plot = calculate_metrics_single(df_sample.copy())

                res = calculate_metrics_single(df_sample)
                self.dist_stats['Random-A']['zero_fwcs'].append((res['FWCS']==0).mean()*100)
                self.dist_stats['Random-A']['gini'].append(self._gini(res['FWCS']))
                self.dist_stats['Random-A']['mean_nd'].append(res['ND'].mean())
                self.dist_stats['Random-A']['n_samples'].append(len(res))

                if (i + 1) % 10 == 0: self._save_stats()
            self._save_stats()

        # --- Random-B ---
        current_len_b = len(self.dist_stats['Random-B']['mean_nd'])
        remaining_b = self.n_resamples - current_len_b

        if remaining_b > 0:
            print(f"   -> Resuming Random-B: {current_len_b} done, {remaining_b} to go.")
            for i in tqdm(range(remaining_b), desc="Random-B"):
                seed_val = 1000 + current_len_b + i
                df_sample = sample_stratified(self.df_f, self.df_r, seed=seed_val)

                if i == 0 and current_len_b == 0:
                    self.df_rand_b_plot = calculate_metrics_single(df_sample.copy())

                res = calculate_metrics_single(df_sample)
                self.dist_stats['Random-B']['zero_fwcs'].append((res['FWCS']==0).mean()*100)
                self.dist_stats['Random-B']['gini'].append(self._gini(res['FWCS']))
                self.dist_stats['Random-B']['mean_nd'].append(res['ND'].mean())
                self.dist_stats['Random-B']['n_samples'].append(len(res))

                if (i + 1) % 10 == 0: self._save_stats()
            self._save_stats()

    def _gini(self, s):
        v = np.sort(s.values); n = len(v)
        if n == 0 or np.sum(v) == 0: return 0.0
        return (np.sum((2 * np.arange(1, n + 1) - n - 1) * v)) / (n * np.sum(v))

    def run_sensitivity_check(self):
        print("\n--- 4. Sensitivity Analysis (Appendix A) ---")
        cache_sens = os.path.join(CACHE_DIR, 'sensitivity_results_v20.pkl')

        if os.path.exists(cache_sens):
            with open(cache_sens, 'rb') as f:
                res = pickle.load(f)
            self._write_sens_txt(res['rho_nd'], res['rho_fwcs'])
            return

        print("   -> Calculating ED<=2 metrics...")
        df_target = self.res_r.copy()
        forms = df_target['form'].values
        freqs = df_target['frequency'].values
        n = len(df_target)

        corpus_forms = self.df_f['form'].values
        corpus_freqs = self.df_f['frequency'].values
        corpus_map = {}
        for f, fr in zip(corpus_forms, corpus_freqs):
            l = len(f)
            if l not in corpus_map: corpus_map[l] = []
            corpus_map[l].append((f, fr))

        nds_2 = np.zeros(n, dtype=int)
        snfs_2 = np.zeros(n, dtype=float)

        for i in tqdm(range(n), desc="ED<=2"):
            t = forms[i]
            candidates = []
            for l in range(len(t) - 2, len(t) + 3):
                if l in corpus_map: candidates.extend(corpus_map[l])
            for cand_f, cand_freq in candidates:
                if t == cand_f: continue
                if Levenshtein.distance(t, cand_f) <= 2:
                    nds_2[i] += 1
                    snfs_2[i] += cand_freq

        fwcs_2 = freqs * nds_2 * snfs_2
        rho_nd, p_nd = stats.spearmanr(df_target['ND'], nds_2)
        rho_fwcs, p_fwcs = stats.spearmanr(df_target['FWCS'], fwcs_2)

        res = {'rho_nd': rho_nd, 'rho_fwcs': rho_fwcs}
        with open(cache_sens, 'wb') as f:
            pickle.dump(res, f)

        self._write_sens_txt(rho_nd, rho_fwcs)

    def _write_sens_txt(self, rho_nd, rho_fwcs):
        print(f"\n[Appendix A Results]")
        print(f"Spearman's Rho (ND, ED=1 vs 2): {rho_nd:.3f}")
        print(f"Spearman's Rho (FWCS, ED=1 vs 2): {rho_fwcs:.3f}")
        with open(os.path.join(OUTPUT_DIR, 'Appendix_Sensitivity.txt'), 'w') as f:
            f.write(f"ND Rho: {rho_nd:.3f}\nFWCS Rho: {rho_fwcs:.3f}\n")

    def analyze_diff_freq_top(self):
        print("\n--- 5. Freq-Top Diff Analysis (Reviewer 3.8) ---")
        set_r = set(self.res_r['orth'].unique())
        df_diff = self.res_top[~self.res_top['orth'].isin(set_r)].copy()
        df_diff = df_diff.sort_values('frequency', ascending=False)
        top_20_diff = df_diff.head(20)[['orth', 'form', 'frequency', 'ND', 'FWCS']]
        if 'pos_raw' in self.df_f.columns:
            pos_map = self.df_f.drop_duplicates('orth').set_index('orth')['pos_raw'].to_dict()
            top_20_diff['POS'] = top_20_diff['orth'].map(pos_map)
        top_20_diff.to_csv(os.path.join(OUTPUT_DIR, 'FreqTop_Diff_Analysis.csv'), index=False)

    def generate_outputs(self):
        print("\n--- 6. Generating Outputs (5 Conditions) ---")

        # Table 1
        if 'pos_raw' in self.res_r.columns:
            pos_map = {
                '명': 'Noun (명)', '동': 'Verb (동)', '형': 'Adjective (형)',
                '부': 'Adverb (부)', '관': 'Determiner (관)', '의': 'Bound Noun (의)',
                '조': 'Particle (조)', '감': 'Interjection (감)', '수': 'Numeral (수)', '대': 'Pronoun (대)'
            }
            def map_pos(p): return pos_map.get(str(p).strip(), f"Other ({p})")
            pos_counts = self.res_r['pos_raw'].apply(map_pos).value_counts()
            pos_pct = self.res_r['pos_raw'].apply(map_pos).value_counts(normalize=True)*100
            df_table1 = pd.DataFrame({'POS Category': pos_counts.index, 'Count': pos_counts.values, 'Percentage': pos_pct.values})
            df_table1.to_csv(os.path.join(OUTPUT_DIR, 'Table1_DescriptiveStats.csv'), index=False)

        # Table 3
        rows = []
        if self.res_full is not None:
            rows.append({
                'Condition': 'Full Lexicon',
                'N': len(self.res_full),
                '% Zero FWCS': f"{(self.res_full['FWCS']==0).mean()*100:.1f}",
                'Gini': f"{self._gini(self.res_full['FWCS']):.4f}",
                'Mean ND': f"{self.res_full['ND'].mean():.2f}"
            })

        for label, df in [('Restricted', self.res_r), ('Freq-Top', self.res_top)]:
            rows.append({
                'Condition': label,
                'N': len(df),
                '% Zero FWCS': f"{(df['FWCS']==0).mean()*100:.1f}",
                'Gini': f"{self._gini(df['FWCS']):.4f}",
                'Mean ND': f"{df['ND'].mean():.2f}"
            })

        for label in ['Random-A', 'Random-B']:
            stats = self.dist_stats[label]
            if len(stats['mean_nd']) > 0:
                m_nd = np.mean(stats['mean_nd']); ci_nd = 1.96 * np.std(stats['mean_nd'])
                mean_n = np.mean(stats['n_samples'])
                std_n = np.std(stats['n_samples'])
                n_str = f"{mean_n:.1f}" if std_n > 0 else f"{int(mean_n)}"

                rows.append({
                    'Condition': label,
                    'N': n_str,
                    '% Zero FWCS': f"{np.mean(stats['zero_fwcs']):.1f} ± {1.96*np.std(stats['zero_fwcs']):.1f}",
                    'Gini': f"{np.mean(stats['gini']):.4f} ± {1.96*np.std(stats['gini']):.4f}",
                    'Mean ND': f"{m_nd:.2f} ± {ci_nd:.2f}"
                })

        df_t = pd.DataFrame(rows)
        print("--- Table 3 (Main Results) ---")
        print(df_t.to_markdown(index=False))
        df_t.to_csv(os.path.join(OUTPUT_DIR, 'Table3_MainResults.csv'), index=False)

        # Figures: Prepare Data
        # Ensure Random samples are available for plotting even if loop didn't run
        if self.df_rand_a_plot is None:
             print("   -> Plotting Check: Generating Random-A sample for plotting...")
             self.df_rand_a_plot = calculate_metrics_single(self.df_f.sample(n=len(self.res_r)).copy())
        if self.df_rand_b_plot is None:
             print("   -> Plotting Check: Generating Random-B sample for plotting...")
             self.df_rand_b_plot = calculate_metrics_single(sample_stratified(self.df_f, self.df_r, seed=1000))

        # Define Configurations (5 Conditions)
        # Style update: Full Lexicon -> Solid Black ('k', '-')
        plot_configs = [
            ('Restricted', self.res_r, 'blue', '-'),
            ('Freq-Top', self.res_top, 'red', '--'),
            ('Random-A', self.df_rand_a_plot, 'green', '-.'),
            ('Random-B', self.df_rand_b_plot, 'purple', '-.')
        ]

        if self.res_full is not None:
            # Insert at beginning
            plot_configs.insert(0, ('Full Lexicon', self.res_full, 'black', '-')) # Solid Black
        else:
            print("   [CRITICAL WARNING] Full Lexicon missing. Skipping in plots.")

        # Plot All Figures
        print(f"   -> Generating Figures with {len(plot_configs)} conditions...")
        self._plot_kde([(l, df['FWCS'], c, s) for l, df, c, s in plot_configs],
                       'Figure 3: Frequency-Weighted Competition Score (FWCS)', 'Log10(FWCS+1)', 'Figure3_FWCS.png', log_x=True)
        self._plot_kde([(l, df['frequency'], c, s) for l, df, c, s in plot_configs],
                       'Figure 1: Log-Frequency Distribution', 'Log10(Frequency)', 'Figure1_Frequency.png', log_x=True)
        self._plot_kde([(l, df['ND'], c, s) for l, df, c, s in plot_configs],
                       'Figure 2: Neighborhood Density (ND)', 'Neighbors (ND)', 'Figure2_ND.png', log_x=False)
        self._plot_kde([(l, df['SNF'], c, s) for l, df, c, s in plot_configs],
                       'Appendix Figure: SNF', 'Log10(SNF)', 'Figure_Appendix_SNF.png', log_x=True)
        print("   -> All Figures Generated Successfully.")

    def _plot_kde(self, data, title, xlabel, filename, log_x):
        plt.figure(figsize=(10, 6))
        for l, d, c, s in data:
            try:
                # IMPORTANT: Drop NA to prevent plotting errors
                clean_d = d.dropna()
                if len(clean_d) > 0:
                    print(f"      - Plotting {l}...", end="")
                    # Use linewidth=2.5 for Full Lexicon (black) to make it stand out
                    lw = 2.5 if c == 'black' else 2.0
                    sns.kdeplot(np.log10(clean_d+1) if log_x else clean_d, label=l, color=c, linestyle=s, fill=False, linewidth=lw, warn_singular=False)
                    print(" OK")
                else:
                    print(f"      - Skipping {l} (Empty Data)")
            except Exception as e:
                print(f"      - [Error] Failed {l}: {e}")

        plt.title(title); plt.xlabel(xlabel); plt.ylabel('Density'); plt.legend(loc='best'); plt.grid(True, alpha=0.3); plt.tight_layout()
        plt.savefig(os.path.join(OUTPUT_DIR, filename), dpi=300); plt.close()

if __name__ == "__main__":
    analyzer = LexicalAnalyzer(*get_base_lexicons(), n_resamples=100)
    analyzer.run_baselines()
    analyzer.run_resampling()
    analyzer.run_sensitivity_check()
    analyzer.analyze_diff_freq_top()
    analyzer.generate_outputs()

>>> [Cache Hit] Loading Lemma Frequency List...
[Full Lemma] Preprocessing...


[Full Lemma] G2P:   0%|          | 0/11185 [00:00<?, ?it/s]

[Full Lemma] Final N=11083
>>> Loading Restricted Lexicon from: /content/drive/MyDrive/Korean_Lexical_Analysis_v14/inputs/basic_vocab_5965.csv
[Restricted] Preprocessing...


[Restricted] G2P:   0%|          | 0/5965 [00:00<?, ?it/s]

[Restricted] Final N=5511

--- 1. Computing Baselines ---
   -> [Cache Hit] Loaded Full Lexicon

--- 2. Resampling (Target N=100) ---

--- 4. Sensitivity Analysis (Appendix A) ---

[Appendix A Results]
Spearman's Rho (ND, ED=1 vs 2): 0.644
Spearman's Rho (FWCS, ED=1 vs 2): 0.693

--- 5. Freq-Top Diff Analysis (Reviewer 3.8) ---

--- 6. Generating Outputs (5 Conditions) ---
--- Table 3 (Main Results) ---
| Condition    |     N | % Zero FWCS   | Gini            | Mean ND     |
|:-------------|------:|:--------------|:----------------|:------------|
| Full Lexicon | 11083 | 36.6          | 0.9962          | 3.84        |
| Restricted   |  5511 | 50.8          | 0.9957          | 1.88        |
| Freq-Top     |  5511 | 36.7          | 0.9929          | 3.66        |
| Random-A     |  5511 | 48.2 ± 1.3    | 0.9963 ± 0.0018 | 1.90 ± 0.10 |
| Random-B     |  5394 | 49.9 ± 0.9    | 0.9949 ± 0.0004 | 2.01 ± 0.09 |
   -> Plotting Check: Generating Random-A sample for plotting...
   -> Plotting Ch